# Create Combined All Data - Fixed Quarter Dates

In [192]:
# Whether this is a sample/debug run or not
DEBUG = False

In [193]:
# Packages
import pandas as pd
import openpyxl

In [194]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv
cr_and_dates = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Credit_Rating\credit_ratings_on_fixed_quarter_dates_with_earnings_call_date.csv')
print(cr_and_dates)

     ticker          fixed_quarter_date   earnings_call_date Rating  \
0      AAPL  2014-07-01 00:00:00.000000  2014-04-23 20:33:08     AA   
1      AAPL  2014-10-01 00:00:00.000000  2014-07-22 22:26:03     AA   
2      AAPL  2015-01-01 00:00:00.000000  2014-10-20 23:20:12     AA   
3      AAPL  2015-04-01 00:00:00.000000  2015-01-27 20:38:04     AA   
4      AAPL  2015-07-01 00:00:00.000000  2015-04-27 20:48:04     AA   
...     ...                         ...                  ...    ...   
7975    ZTS  2015-10-01 00:00:00.000000  2015-08-04 15:19:08    BBB   
7976    ZTS  2016-01-01 00:00:00.000000  2015-11-03 14:45:11    BBB   
7977    ZTS  2016-04-01 00:00:00.000000  2016-02-16 16:13:08    BBB   
7978    ZTS  2016-07-01 00:00:00.000000  2016-05-04 15:49:11    BBB   
7979    ZTS  2016-10-01 00:00:00.000000  2016-08-03 16:27:41    BBB   

                      Rating Agency Name rating_date         Source  \
0     Standard & Poor's Ratings Services  2014-05-27  Supplementary   
1    

In [195]:
# check for duplicates on ticker by fixed_quarter_date
print(cr_and_dates[cr_and_dates.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call]
Index: []


In [196]:
# Load ~\Box\STAT 222 Capstone\Intermediate Data\Calls\calls.csv
if DEBUG:
    calls = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\calls.csv', nrows = 10)
else:
    calls = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Calls\combined_calls.csv')
# Delete the first column (index)
#calls = calls.iloc[:,1:]
# rename symbol to company
calls = calls.rename(columns = {'symbol':'company'})
# Convert date to be just the date part
calls['date'] = calls['date'].str[:10]
print(calls)

      company  quarter  year        date  \
0           A        3  2011  2011-02-15   
1           A        4  2011  2011-05-13   
2           A        1  2012  2011-08-16   
3           A        2  2012  2011-11-16   
4           A        3  2012  2012-02-16   
...       ...      ...   ...         ...   
31067     ZTS        4  2015  2016-02-16   
31068     ZTS        1  2016  2016-05-04   
31069     ZTS        2  2016  2016-08-03   
31070     ZTS        3  2016  2016-11-02   
31071     ZTS        4  2016  2017-02-16   

                                                 content  source    web  
0      Operator: Good day, ladies and gentlemen, and ...  kaggle  False  
1      Operator: Good day, ladies and gentlemen, and ...  kaggle  False  
2      Operator: Good day, ladies and gentlemen, and ...  kaggle  False  
3      Operator: Good day, ladies and gentlemen, and ...  kaggle  False  
4      Operator: Good day, ladies and gentlemen, and ...  kaggle  False  
...                        

In [197]:
# check for duplicates on company by date
print(calls[calls.duplicated(subset=['company', 'date'], keep=False)])

Empty DataFrame
Columns: [company, quarter, year, date, content, source, web]
Index: []


### Earnings Call Data Merge

In [198]:
# convert earnings_call_date to date part
cr_and_dates['earnings_call_date'] = cr_and_dates['earnings_call_date'].str[:10]

# Inner join cr_and_dates (on columns ticker, earnings_call_date) with calls.csv (on columns company, date)
merged = pd.merge(cr_and_dates, calls, left_on=['ticker', 'earnings_call_date'], right_on=['company', 'date'], how='inner')
merged

# Print range of dates
print('date range')
print(merged['date'].min())
print(merged['date'].max())

# Range of year, credit_rating_year
for variable in ['year', 'credit_rating_year']:
    print(variable)
    print(merged[variable].min())
    print(merged[variable].max())

# Print rows where year is greater than 2016
# This is due to inconsistencies in the earnings call data year variable
print('years greater than 2016')
print(merged[merged['year'] > 2016][['earnings_call_date', 'ticker', 'company', 'date', 'year', 'credit_rating_year']])

# Free up memory from calls
del calls

date range
2010-05-03
2016-09-30
year
2010
2016
credit_rating_year
2010
2016
years greater than 2016
Empty DataFrame
Columns: [earnings_call_date, ticker, company, date, year, credit_rating_year]
Index: []


In [199]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call, company, quarter, year, date, content, source, web]
Index: []

[0 rows x 23 columns]


In [200]:
# Rename content to transcript
merged = merged.rename(columns = {'content':'transcript'})

### Financial Statements Data Merge

In [201]:
# Load tabular financial statement data
df = pd.read_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\Tabular_Fin\combined_corrected_tabular_financial_statements_data.parquet')
print(df)

# Perform merge
merged = pd.merge(merged, df, left_on=['company', 'year', 'quarter'], right_on=['symbol', 'calendarYear', 'period'], how='inner')    
print(merged)

            date symbol reportedCurrency      cik fillingDate  \
0     2016-12-31    BEP              USD  1533232  2017-03-01   
1     2016-09-30    BEP              USD  1533232  2016-10-03   
2     2016-06-30    BEP              USD  1533232  2016-07-05   
3     2016-03-31    BEP              USD  1533232  2016-04-01   
4     2015-12-31    BEP              USD  1533232  2016-01-04   
...          ...    ...              ...      ...         ...   
20820 2011-03-31   YORW              USD   108985  2011-05-06   
20821 2010-12-31   YORW              USD   108985  2011-03-08   
20822 2010-09-30   YORW              USD   108985  2010-11-05   
20823 2010-06-30   YORW              USD   108985  2010-08-06   
20824 2010-03-31   YORW              USD   108985  2010-05-07   

              acceptedDate  calendarYear  period  cashAndCashEquivalents  \
0      2017-03-01 06:27:55          2016       4             223000000.0   
1      2016-10-03 06:45:16          2016       3             232000

In [202]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call, company, quarter, year, date_x, transcript, source, web, date_y, symbol, reportedCurrency, cik, fillingDate, acceptedDate, calendarYear, period, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabilit

### Rating Changes

In [203]:
# Variable for rating on previous fixed_quarter_date
# Sort by ticker, fixed_quarter_date
merged = merged.sort_values(by=['ticker', 'fixed_quarter_date'])
# Shift the rating by 1
merged['rating_on_previous_fixed_quarter_date'] = merged.groupby('ticker')['Rating'].shift(1)
merged[['ticker', 'fixed_quarter_date', 'Rating', 'rating_on_previous_fixed_quarter_date']].head(10)

,ticker,fixed_quarter_date,Rating,rating_on_previous_fixed_quarter_date
0,AAPL,2014-07-01 00:00:00.000000,AA,NaN
1,AAPL,2014-10-01 00:00:00.000000,AA,AA
2,AAPL,2015-01-01 00:00:00.000000,AA,AA
3,AAPL,2015-04-01 00:00:00.000000,AA,AA
4,AAPL,2015-07-01 00:00:00.000000,AA,AA
5,AAPL,2015-10-01 00:00:00.000000,AA,AA
6,AAPL,2016-01-01 00:00:00.000000,AA,AA
7,AAPL,2016-04-01 00:00:00.000000,AA,AA
8,AAPL,2016-07-01 00:00:00.000000,AA,AA
9,AAPL,2016-10-01 00:00:00.000000,AA,AA


In [204]:
# Rating change variables
# Get numeric values for Rating and rating_on_next_earnings_call_date
# Encode Ratings as ordinal
rating_order = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'CC', 'C', 'D']
# Reverse
rating_order = rating_order[::-1]
# Create dictionary mapping to number values
rating_dict = {rating: i for i, rating in enumerate(rating_order, 1)}
# Create new column "Rating Rank AAA is 10"
merged['Rating Rank AAA is 10'] = merged['Rating'].map(rating_dict)
# Flag for investment grade - Rating Rank >= 7.0
merged['Investment_Grade'] = merged['Rating Rank AAA is 10'] >= 7.0
# Create new column "rating_on_previous_fixed_quarter_date AAA is 10"
merged['rating_on_previous_fixed_quarter_date AAA is 10'] = merged['rating_on_previous_fixed_quarter_date'].map(rating_dict)

# Cross tab Rating and 'Rating Rank AAA is 10'
print(pd.crosstab(merged['Rating'], merged['Rating Rank AAA is 10']))
# Cross tab rating_on_previous_fixed_quarter_date and 'rating_on_previous_fixed_quarter_date AAA is 10'
print(pd.crosstab(merged['rating_on_previous_fixed_quarter_date'], merged['rating_on_previous_fixed_quarter_date AAA is 10']))

Rating Rank AAA is 10  1   2   3    4     5     6     7     8    9    10
Rating                                                                  
A                       0   0   0    0     0     0     0  1388    0    0
AA                      0   0   0    0     0     0     0     0  339    0
AAA                     0   0   0    0     0     0     0     0    0  137
B                       0   0   0    0  1143     0     0     0    0    0
BB                      0   0   0    0     0  1870     0     0    0    0
BBB                     0   0   0    0     0     0  2232     0    0    0
C                       0  16   0    0     0     0     0     0    0    0
CC                      0   0  14    0     0     0     0     0    0    0
CCC                     0   0   0  185     0     0     0     0    0    0
D                       9   0   0    0     0     0     0     0    0    0
rating_on_previous_fixed_quarter_date AAA is 10  1.0   2.0   3.0   4.0   5.0   \
rating_on_previous_fixed_quarter_date      

In [205]:
# Variable for change before next call
def change(row):
    if row['Rating Rank AAA is 10'] < row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Downgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] > row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Upgrade Since Last Fixed Quarter Date'
    elif row['Rating Rank AAA is 10'] == row['rating_on_previous_fixed_quarter_date AAA is 10']:
        return 'Same As Last Fixed Quarter Date'
    else:
        return None

# Apply function to create new column
merged['Change Direction Since Last Fixed Quarter Date'] = merged.apply(change, axis = 1)

# Also numeric version 'Change Since Last Fixed Quarter Date'
merged['Change Since Last Fixed Quarter Date'] = merged['Rating Rank AAA is 10'] - merged['rating_on_previous_fixed_quarter_date AAA is 10']

# Cross tab 'Change Direction Since Last Fixed Quarter Date' and 'Change Since Last Fixed Quarter Date'
print(pd.crosstab(merged['Change Direction Since Last Fixed Quarter Date'], merged['Change Since Last Fixed Quarter Date']))

Change Since Last Fixed Quarter Date            -2.0  -1.0   0.0   1.0   2.0
Change Direction Since Last Fixed Quarter Date                              
Downgrade Since Last Fixed Quarter Date           17   149     0     0     0
Same As Last Fixed Quarter Date                    0     0  6432     0     0
Upgrade Since Last Fixed Quarter Date              0     0     0   180    19


### Sector and Other Textual Company Information

From Kaggle: https://www.kaggle.com/datasets/aramacus/usa-public-companies

Supplemented with manually looked up sector data for missing sectors

In [206]:
# Load CSV 'C:\Users\ijyli\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv'
combined_sector_data = pd.read_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\Sectors\combined_sector_data.csv')
print(combined_sector_data)

     Ticker                                        Description  \
0      CTVA  Corteva, Inc. is a global provider of seed and...   
1      ALCO  Alico, Inc. is an agribusiness and land manage...   
2      LMNR  Limoneira Company is primarily an agribusiness...   
3      SANW  S&W Seed Company (S&W) is a multi-crop and mid...   
4       TRC  Tejon Ranch Co. is a diversified real estate d...   
...     ...                                                ...   
3384   RELX                                                NaN   
3385    TAL                                                NaN   
3386   TRTN                                                NaN   
3387   VLRS                                                NaN   
3388    XIN                                                NaN   

        Company Name                  Sector            Industry Group  \
0        CORTEVA INC               Materials                 Materials   
1          ALICO INC        Consumer Staples  Food, Beverag

In [207]:
# Left join us_stocks with merged
merged = pd.merge(merged, combined_sector_data, left_on='ticker', right_on='Ticker', how='left')
print(merged)

     ticker          fixed_quarter_date earnings_call_date Rating  \
0      AAPL  2014-07-01 00:00:00.000000         2014-04-23     AA   
1      AAPL  2014-10-01 00:00:00.000000         2014-07-22     AA   
2      AAPL  2015-01-01 00:00:00.000000         2014-10-20     AA   
3      AAPL  2015-04-01 00:00:00.000000         2015-01-27     AA   
4      AAPL  2015-07-01 00:00:00.000000         2015-04-27     AA   
...     ...                         ...                ...    ...   
7328    ZTS  2015-10-01 00:00:00.000000         2015-08-04    BBB   
7329    ZTS  2016-01-01 00:00:00.000000         2015-11-03    BBB   
7330    ZTS  2016-04-01 00:00:00.000000         2016-02-16    BBB   
7331    ZTS  2016-07-01 00:00:00.000000         2016-05-04    BBB   
7332    ZTS  2016-10-01 00:00:00.000000         2016-08-03    BBB   

                      Rating Agency Name rating_date         Source  \
0     Standard & Poor's Ratings Services  2014-05-27  Supplementary   
1     Standard & Poor's Ratin

In [208]:
# check for duplicates on ticker by fixed_quarter_date
print(merged[merged.duplicated(subset=['ticker', 'fixed_quarter_date'], keep=False)])

Empty DataFrame
Columns: [ticker, fixed_quarter_date, earnings_call_date, Rating, Rating Agency Name, rating_date, Source, Rating Rank AAA is 10, Next Rating, Next Rating Date, Previous Rating, Previous Rating Date, next_rating_date_or_end_of_data, credit_rating_year, previous_fixed_quarter_date, days_since_call, company, quarter, year, date_x, transcript, source, web, date_y, symbol, reportedCurrency, cik, fillingDate, acceptedDate, calendarYear, period, cashAndCashEquivalents, shortTermInvestments, cashAndShortTermInvestments, netReceivables, inventory, otherCurrentAssets, totalCurrentAssets, propertyPlantEquipmentNet, goodwill, intangibleAssets, goodwillAndIntangibleAssets, longTermInvestments, taxAssets, otherNonCurrentAssets, totalNonCurrentAssets, otherAssets, totalAssets, accountPayables, shortTermDebt, taxPayables, deferredRevenue, otherCurrentLiabilities, totalCurrentLiabilities, longTermDebt, deferredRevenueNonCurrent, deferredTaxLiabilitiesNonCurrent, otherNonCurrentLiabilit

In [209]:
# Print if ticker = PCAR and fixed_quarter_date = 2011-07-01
# Compare fixed_quarter_date to datetime of 2011-07-01
#print(merged[(merged['ticker'] == 'PCAR') & (merged['fixed_quarter_date'] == pd.to_datetime('2011-07-01'))])

### Some Basic Stats

In [210]:
# Print range of dates
# print('date range')
# print(merged['date'].min())
# print(merged['date'].max())

# Range of year, credit_rating_year, calendarYear
# for variable in ['year', 'credit_rating_year', 'calendarYear']:
#     print(variable)
#     print(merged[variable].min())
#     print(merged[variable].max())

In [211]:
# Summarize columns
print(merged.info())
# print column names
print(merged.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7333 entries, 0 to 7332
Columns: 170 entries, ticker to Comment
dtypes: bool(2), datetime64[ns](1), float64(114), int32(1), int64(9), object(43)
memory usage: 9.4+ MB
None
Index(['ticker', 'fixed_quarter_date', 'earnings_call_date', 'Rating',
       'Rating Agency Name', 'rating_date', 'Source', 'Rating Rank AAA is 10',
       'Next Rating', 'Next Rating Date',
       ...
       'Change Direction Since Last Fixed Quarter Date',
       'Change Since Last Fixed Quarter Date', 'Ticker', 'Description',
       'Company Name', 'Sector', 'Industry Group', 'Industry', 'Sub-Industry',
       'Comment'],
      dtype='object', length=170)


In [212]:
# Save a parquet file as well
merged.to_parquet(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates.parquet', index=False)

In [213]:
# Sample 100 rows to create all_data_sample.csv
if not DEBUG:
    all_data_sample = merged.sample(100)
    all_data_sample.to_csv(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.csv', index=False)
    # also save to xlsx
    all_data_sample.to_excel(r'~\Box\STAT 222 Capstone\Intermediate Data\All_Data\all_data_fixed_quarter_dates_sample.xlsx', index=False)